<h1>JanataHack Steam Reviews Sentiment Analysis Hackathon</h1>
<h2>Approach 1 - Count Features and XGBoost</h2>
Based on the frequency of occurence of words,symbols and digits in positive and negative reviews.
Got 0.82 F1-score on the leaderboard.

<h3>Importing relevant libraries</h3>

In [1]:
from time import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


/home/hithesh/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/hithesh/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


<h3>Loading train and test datasets as dataframes</h3>

In [2]:
df = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/train.csv')
df_test = pd.read_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/test.csv')

df['length'] = df['user_review'].apply(lambda x: len(x))
df_test['length'] = df_test['user_review'].apply(lambda x: len(x))
df.head()

,review_id,title,year,user_review,user_suggestion,length
0,1,Spooky's Jump Scare Mansion,2016.0,I'm scared and hearing creepy voices. So I'll...,1,710
1,2,Spooky's Jump Scare Mansion,2016.0,"Best game, more better than Sam Pepper's YouTu...",1,335
2,3,Spooky's Jump Scare Mansion,2016.0,"A littly iffy on the controls, but once you kn...",1,397
3,4,Spooky's Jump Scare Mansion,2015.0,"Great game, fun and colorful and all that.A si...",1,280
4,5,Spooky's Jump Scare Mansion,2015.0,Not many games have the cute tag right next to...,1,334


<h3>Combining reviews to make documents.</h3>
Creating a document of all reviews in the training data, another document of all reviews in which the user recommends the game and another one which has all reviews in which user doesn't recommend the game

In [3]:
doc = ''
doc1 = ''
doc0 = ''
for i in range(df.shape[0]):
    doc = doc + df.loc[i,'user_review'] + str(' ')
    if(df.loc[i,'user_suggestion']==0):
        doc0 = doc0 + df.loc[i,'user_review']
    else:
        doc1 = doc1 + df.loc[i,'user_review']
n = 0

<h3>Preprocessing the text in the three documents</h3>
Removing stopwords (words which don't add relevant amount of meaning to sentences), keeping only alphabet characters and converting documents to lowercase. 

In [4]:
stop_words = stopwords.words('english')

tokens = word_tokenize(doc)
all_words = [token.lower() for token in tokens if token.isalpha()!=0]
all_words = [word for word in all_words if word not in stop_words]

tokens0 = word_tokenize(doc0)
all_words0 = [token.lower() for token in tokens0 if token.isalpha()!=0]
all_words0 = [word for word in all_words0 if word not in stop_words]

tokens1 = word_tokenize(doc1)
all_words1 = [token.lower() for token in tokens1 if token.isalpha()!=0]
all_words1 = [word for word in all_words1 if word not in stop_words]

<h3>Getting frequency distributions of all words in the document</h3>
The most common words in the three documents and their frequencies are given below, first being the whole document, 2nd, the positive reviews and 3rd, the negative reviews.

In [5]:
freq_all_words = FreqDist(all_words)
freq_all_words0 = FreqDist(all_words0)
freq_all_words1 = FreqDist(all_words1)
freq_all_words.most_common(15), '\n',freq_all_words1.most_common(15),'\n',freq_all_words0.most_common(15)

([('game', 50545),
  ('like', 11852),
  ('play', 11379),
  ('get', 11363),
  ('good', 7490),
  ('time', 7205),
  ('early', 6889),
  ('access', 6887),
  ('one', 6835),
  ('fun', 6733),
  ('even', 6337),
  ('really', 6054),
  ('would', 5436),
  ('games', 5308),
  ('free', 5019)],
 '\n',
 [('game', 27100),
  ('like', 7116),
  ('play', 6369),
  ('get', 5762),
  ('good', 4471),
  ('fun', 4115),
  ('access', 3907),
  ('one', 3742),
  ('time', 3627),
  ('really', 3619),
  ('free', 3451),
  ('games', 3260),
  ('nope', 3229),
  ('would', 2834),
  ('great', 2803)],
 '\n',
 [('game', 22473),
  ('get', 5584),
  ('play', 4900),
  ('like', 4709),
  ('even', 3650),
  ('time', 3376),
  ('one', 2986),
  ('access', 2935),
  ('good', 2894),
  ('money', 2759),
  ('would', 2580),
  ('fun', 2413),
  ('really', 2382),
  ('new', 2242),
  ('pay', 2167)])

<h3>Creating word frequency features</h3>
Having words in the top 1000 most common words in the negative review document which are not in top 1000 of most common words in the positive review document along with 300 most common words in the full document is found to obtain a higher evaluation score.

In [6]:
word_features = []
word_features0 = []
word_features1 = []

for i in range(1000):
    word_features0.append(freq_all_words0.most_common(1000)[i][0])
for i in range(1000):
    if(freq_all_words1.most_common(1000)[i][0] in word_features0):
        word_features0.remove(freq_all_words1.most_common(1000)[i][0])
        
for i in range(300):
    word_features.append(freq_all_words.most_common(300)[i][0])

word_features =  word_features + word_features0 + word_features1
word_features = list(set(word_features))
len(word_features)

493

<h3>Preparing training data to feed the model</h3>
Getting counts of occurences of words, symbols, letters and digits as training and testing examples

In [7]:
for w in word_features:
    df[w] = df['user_review'].apply(lambda x: len(x.lower().split(w))-1)

symbol_features = ['!','.','\'',',','@','$','#',':',';','\"','&','*','%','^',' ']
for s in symbol_features:
    df[s] = df['user_review'].apply(lambda x: len(x.split(s))-1)

alpha_features = [chr(i) for i in range(ord('a'),ord('z')+1)]
for f in alpha_features:
    df[f] = df['user_review'].apply(lambda x: len(x.split(f))-1)
    
digit_features = [str(i) for i in range(10)]
for d in digit_features:
    df[d] = df['user_review'].apply(lambda x: len(x.split(d))-1)
    
for w in word_features:
    df_test[w] = df_test['user_review'].apply(lambda x: len(x.lower().split(w))-1)
    
for s in symbol_features:
    df_test[s] = df_test['user_review'].apply(lambda x: len(x.split(s))-1)

alpha_features = [chr(i) for i in range(ord('a'),ord('z')+1)]
for f in alpha_features:
    df_test[f] = df_test['user_review'].apply(lambda x: len(x.lower().split(f))-1)
    
digit_features = [str(i) for i in range(10)]
for d in digit_features:
    df_test[d] = df_test['user_review'].apply(lambda x: len(x.split(d))-1)
    
features = list(set(word_features + symbol_features + alpha_features + digit_features + ['length']))
df.head()

,review_id,title,year,user_review,user_suggestion,length,made,ruined,internet,barely,...,0,1,2,3,4,5,6,7,8,9
0,1,Spooky's Jump Scare Mansion,0,I'm scared and hearing creepy voices. So I'll...,1,710,0,0,0,0,...,1,1,0,0,0,0,0,0,0,2
1,2,Spooky's Jump Scare Mansion,0,"Best game, more better than Sam Pepper's YouTu...",1,335,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
2,3,Spooky's Jump Scare Mansion,0,"A littly iffy on the controls, but once you kn...",1,397,1,0,0,0,...,4,2,0,0,0,1,0,0,0,0
3,4,Spooky's Jump Scare Mansion,0,"Great game, fun and colorful and all that.A si...",1,280,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Spooky's Jump Scare Mansion,0,Not many games have the cute tag right next to...,1,334,0,0,0,0,...,4,2,1,0,1,0,0,0,0,0


<h3>Model Optimization and Evaluation</h3>
Models like Random Forest, Gradient Boost, SVM, Logistic Regression have been tried. XGBoost with n_estimators = 500 was found to obtain the cross validation score.

In [8]:
X = df[features]
y = df['user_suggestion']

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

model=XGBClassifier(n_estimators = 500)
model.fit(X_train,y_train)

train_score = f1_score(y_train, model.predict(X_train))
test_score = f1_score(y_test, model.predict(X_test))
print(train_score,test_score)

0.99985588701542 0.8403252032520325


<h3>Making Predictions</h3>

In [9]:
model.fit(X,y)
test_X = df_test[features]

df_test['user_suggestion'] = pd.Series(model.predict(test_X))
final = pd.concat([df_test['review_id'],df_test['user_suggestion']],axis=1)
final.to_csv('/home/hithesh/Desktop/ML Competitions/Sentiment_Analysis_Analytics_vidhya/nlp_sub1.csv',header=True , index=False)